In [1]:
import logging

import probtorch
import torch

import combinators
import filtering
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.GraphingTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi, pi0 = hmm_run()

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
inference = combinators.GraphingTrace(100)

In [12]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}
    hmm_params['Pi_%d' % k]['concentration'][1] = 1.25
    hmm_params['Pi_%d' % k]['concentration'][2] = 1.5
    hmm_params['Pi_%d' % k]['concentration'][3] = 1.25

In [13]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(inference, generative)

In [14]:
z0, mu, sigma, pi, pi0 = init_hmm(T=50)

In [15]:
hmm_step = hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [16]:
hmm_run.condition(inference, generative)

In [17]:
z_last, mu, sigma, pi, pi0 = hmm_run()

In [18]:
hmm_step.backward_pass(T=50)

In [19]:
hmm_step.smoothed_posterior(T=50)[0][3]

tensor([[  -4.6505,   -0.3037,   -1.3898,   -5.8631,   -7.8483],
        [ -25.5752,   -2.9114,   -0.5975,   -1.0066,   -3.5089],
        [ -10.4240,   -0.9558,   -0.5356,   -3.5679,   -6.2488],
        [  -0.0255,   -3.6830,  -12.7175,  -19.8969,  -26.4652],
        [ -22.6156,   -2.0823,   -0.2835,   -3.4719,   -2.3954],
        [ -41.0435,   -5.5646,   -2.6120,   -0.2037,   -2.2345],
        [  -0.0198,   -3.9351,  -11.2157,  -18.1568,  -24.0659],
        [  -0.4872,   -0.9731,   -4.8655,  -12.8951,  -14.5590],
        [ -21.8135,   -2.2539,   -0.2929,   -2.0839,   -3.7093],
        [  -9.9326,   -0.8283,   -0.6221,   -3.6941,   -6.5271],
        [  -0.3805,   -1.1849,   -4.5432,  -10.1644,  -14.8302],
        [ -10.6887,   -0.9119,   -0.5366,   -4.5040,   -6.0346],
        [ -17.9003,   -1.8074,   -0.3637,   -2.0850,   -4.1044],
        [ -29.3008,   -3.6457,   -0.7762,   -0.7669,   -3.0098],
        [  -3.4682,   -0.2913,   -1.5154,   -6.4933,   -8.2115],
        [ -70.7150,   -9.

In [20]:
init_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.rand(5) * 10,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    init_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [21]:
init_hmm = combinators.Model(f=hmm.init_hmm, phi=init_hmm_params)

In [22]:
def hmm_step_builder(z0, mu, sigma, pi, pi0):
    return hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)

In [23]:
inference, variational_params = filtering.variational_forward_backward(init_hmm, hmm_step_builder, 500, 50, generative, use_cuda=False, lr=1e-2)

07/27/2018 10:12:17 Variational forward-backward ELBO=-6.81163818e+03 at epoch 1
07/27/2018 10:12:17 Variational forward-backward ELBO=-6.15462500e+03 at epoch 2
07/27/2018 10:12:17 Variational forward-backward ELBO=-6.35909033e+03 at epoch 3
07/27/2018 10:12:17 Variational forward-backward ELBO=-6.60342725e+03 at epoch 4
07/27/2018 10:12:17 Variational forward-backward ELBO=-6.01459375e+03 at epoch 5
07/27/2018 10:12:17 Variational forward-backward ELBO=-6.17575293e+03 at epoch 6
07/27/2018 10:12:18 Variational forward-backward ELBO=-6.43789600e+03 at epoch 7
07/27/2018 10:12:18 Variational forward-backward ELBO=-6.50927686e+03 at epoch 8
07/27/2018 10:12:18 Variational forward-backward ELBO=-5.97945361e+03 at epoch 9
07/27/2018 10:12:18 Variational forward-backward ELBO=-6.28020166e+03 at epoch 10
07/27/2018 10:12:18 Variational forward-backward ELBO=-6.86920703e+03 at epoch 11
07/27/2018 10:12:18 Variational forward-backward ELBO=-7.06325586e+03 at epoch 12
07/27/2018 10:12:19 Varia

In [24]:
variational_params

"{'Pi_0__concentration': 'Parameter containing:\ntensor([ 1.0591,  0.6676,  1.6160,  0.7908,  0.7674])', 'Pi_1__concentration': 'Parameter containing:\ntensor([ 1.6476,  1.1072,  1.7426,  1.4217,  0.7498])', 'Pi_2__concentration': 'Parameter containing:\ntensor([ 0.9283,  1.0927,  1.8685,  1.5870,  1.0108])', 'Pi_3__concentration': 'Parameter containing:\ntensor([ 1.2972,  0.6900,  2.1388,  1.2953,  0.4213])', 'Pi_4__concentration': 'Parameter containing:\ntensor([ 0.7973,  0.9014,  1.8657,  1.6362,  1.1792])', 'Pi_5__concentration': 'Parameter containing:\ntensor([ 1.0138,  1.2506,  1.7561,  1.4040,  0.9238])', 'mu__loc': 'Parameter containing:\ntensor([ 1.5305,  5.2141,  3.0557,  4.9786,  5.7935])', 'mu__scale': 'Parameter containing:\ntensor([-0.2798,  0.0811, -0.3821,  0.0032,  0.0375])', 'sigma__loc': 'Parameter containing:\ntensor([ 0.9491,  2.0425,  1.8004,  2.3201,  2.0914])', 'sigma__scale': 'Parameter containing:\ntensor([ 0.9070,  0.5401,  0.2467, -0.0384,  0.6949])'}"